In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import os
from scipy.sparse import csr_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import celltypist
from celltypist import models
import urllib.request
import gzip
import pybedtools
import pickle
sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    color_map="YlGnBu",
    frameon=True,
    fontsize=8,
)
# sns.set(style="whitegrid")
sns.set_style("white", {'axes.grid': False})
plt.rcParams['figure.subplot.wspace'] = 0.3
plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams['font.size'] = 8

res_path = ''
grna = ''
editing_pkl = ''
editing_tsv = ''

os.chdir(res_path)
adata = sc.read_h5ad('adata.h5ad')
adata_editing = sc.read_h5ad('adata_editing.h5ad')


markers = ['DDX39B', 'AIMP1', 'TPR', 'SLTM', 'ILF3', 'ADAR', 'DHX9']
pd.DataFrame(adata[:, markers].X)
adata.obs.loc[:, markers] = pd.DataFrame(
    adata[:, markers].X, index=adata.obs.index, columns=markers)
# The KD expression level need to be lowest 25%
expression_dict_25 = adata.obs.loc[:, markers].quantile(0.25).to_dict()
expression_dict_15 = adata.obs.loc[:, markers].quantile(0.15).to_dict()


def filter_kd(x):
    gene = x.gRNA.split('-')[1]
    if gene == 'Control':
        for marker in markers:
            if x[marker] <= expression_dict_15[marker]:
                return False
        return True
    else:
        if x[f'{gene}'] <= expression_dict_25[gene]:
            return True
        return False


adata.obs['filter_kd'] = adata.obs.apply(filter_kd, axis=1)
adata_editing.obs['filter_kd'] = adata_editing.obs.index.isin(
    adata.obs.index[adata.obs['filter_kd']])

adata_editing = adata_editing[adata_editing.obs['filter_kd']]
adata = adata[adata.obs['filter_kd']]
adata.obs = adata.obs.drop(columns=markers)
grade_order = ['kd-Control-1',
               'kd-Control-2',
               'kd-Control-3',
               'kd-Control-4',
               'kd-ADAR-1',
               'kd-ADAR-2', 'kd-ILF3-1',
               'kd-ILF3-2', 'kd-DHX9-1',
               'kd-DHX9-2',
               'kd-AIMP1-1',
               'kd-AIMP1-2',
               'kd-SLTM-1',
               'kd-SLTM-2',
               'kd-TPR-1',
               'kd-TPR-2',
               'kd-DDX39B-1',
               'kd-DDX39B-2',
               ]
palette = [
    '#717171',
    '#7f7f7f',
    '#a5a5a5',
    '#cbcbcb',
    '#598351',
    '#72a968',
    '#165581',
    '#1f77b4',
    '#ad3c37',
    '#c45b56',
    '#68929e',
    '#a7dfff',
    '#006374',
    '#00839a',
    '#7e789a',
    '#9d95c0',
    '#a66a95',
    '#f29ad9',
]

In [ ]:
adata.obs['ct'] = adata.obs.apply(
    lambda x: 'kd-Control' if 'NT' in x.gene_target else 'kd-' + x.gene_target, axis=1)
sorted(adata.obs.ct.unique())
sc.tl.rank_genes_groups(adata, 'ct', groups=['kd-ADAR',
                                             'kd-ILF3',
                                             'kd-DHX9',
                                             'kd-AIMP1',
                                             'kd-SLTM',
                                             'kd-TPR',
                                             'kd-DDX39B',
                                             ], reference='kd-Control')
# sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)
names = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
scores = pd.DataFrame(adata.uns['rank_genes_groups']['scores'])
pvals = pd.DataFrame(adata.uns['rank_genes_groups']['pvals'])
pvals_adj = pd.DataFrame(adata.uns['rank_genes_groups']['pvals_adj'])
log_fold_changes = pd.DataFrame(
    adata.uns['rank_genes_groups']['logfoldchanges'])
df_dict = {}
for col in names.columns:
    df_dict[col] = pd.concat([names[col], scores[col],
                              pvals[col], pvals_adj[col], log_fold_changes[col]], axis=1)
    df_dict[col].columns = [
        'gene',
        'score',
        'pvals',
        'pvals_adj',
        'log_fold_change']
res_df = []
for k in df_dict.keys():
    df = df_dict[k]
    df = df[(df['pvals_adj'] <= 0.05) & (df['log_fold_change'].abs() >= 1)]
    res_df.append([k, df[(df['pvals_adj'] <= 0.05) & (
        df['log_fold_change'].abs() >= 1)].shape[0], df[(df['pvals_adj'] <= 0.05) & (
            df['log_fold_change'] >= 1)].shape[0], df[(df['pvals_adj'] <= 0.05) & (
                df['log_fold_change'] <= -1)].shape[0]])
res_df = pd.DataFrame(res_df, columns=['group', 'n_genes', 'up', 'down'])
# res_df.set_index('group', inplace=True)
# res_df = res_df.loc[
res_df['down'] = res_df['down'] * -1
# plt.figure(figsize=(3, 2))
sns.barplot(data=res_df, x='group', y='up', palette=palette[4:-1:2])
plt.xticks(rotation=90)
plt.ylabel('Number of DEGs')
plt.ylim(-500, 1200)
plt.xlabel('gRNA')
plt.savefig('figures/DEGs_up.pdf', bbox_inches='tight')

In [5]:
adata = sc.read_h5ad('sc_dimensionality_clustered.h5')
grna_ = pd.read_csv(grna)
grna_.set_index('Cell Barcode', inplace=True)
adata.obs = adata.obs.join(grna_, how='left')
adata.obs['gRNA_raw'] = adata.obs.gRna.copy()
dict_grna_name = {}


def get_grna_name(x):
    if pd.isna(x):
        return 'Unassigned'
    gene = x.split('_')[0]
    gene = gene.split('-')[0]
    if gene == 'non':
        gene = 'Control'
    if gene not in dict_grna_name:
        dict_grna_name[gene] = {}
    if x not in dict_grna_name[gene]:
        dict_grna_name[gene][x] = str(len(dict_grna_name[gene]) + 1)
    return 'kd-' + gene + '-' + dict_grna_name[gene][x]


adata.obs['gRNA'] = adata.obs.gRNA_raw.apply(get_grna_name)
sc.pl.umap(
    adata,
    color=[
        "total_counts",
        'scrublet_class',
        'leiden_res0_5'],
    save='_res_umap_with_scrublet.pdf', show=True)
sc.pl.umap(adata[~adata.obs.scrublet_class], color=[
           "total_counts", 'scrublet_class', 'leiden_res0_5'], save='_res_umap_without_scrublet.pdf', show=True)
sc.pl.tsne(
    adata,
    color=[
        "total_counts",
        'scrublet_class',
        'leiden_res0_5'],
    save='_res_tsne_with_scrublet.pdf', show=True)
sc.pl.tsne(adata[~adata.obs.scrublet_class], color=[
           "total_counts", 'scrublet_class', 'leiden_res0_5'], save='_res_tsne_without_scrublet.pdf', show=True)
markers = ['DDX39B', 'AIMP1', 'TPR', 'SLTM', 'ILF3', 'ADAR', 'DHX9']
adata = adata[~adata.obs.scrublet_class]
print(adata.obs[~(adata.obs.gRNA == 'Unassigned')
                ].shape[0] / adata.obs.shape[0])
adata = adata[~(adata.obs.gRNA == 'Unassigned')]
adata.X = adata.layers['raw'].copy()

In [ ]:
adata.obs[adata.obs.gRNA_group == 'kd-ADAR'].mixscape_class_p_ko.describe()

In [ ]:
import gzip
import pickle
from single_cell.single_cell_editing import res_to_edited_matrix
from matplotlib import pyplot as plt
with gzip.open('known_editing_site.pickle.gz', 'rb') as f:
    known_editing_site = pickle.load(f)
with gzip.open(editing_pkl, 'rb') as f:
    editing_pickle = pickle.load(f)
filter_res = pd.read_csv(
    editing_tsv,
    sep='\t')
filter_res = filter_res.drop_duplicates()
filter_res = filter_res[filter_res['FILTER_PASS']]
filter_res['index'] = filter_res['CHROM'] + ':' + \
    filter_res['POS'].astype(str) + ':' + '.'
filter_res.set_index('index', inplace=True)
filter_res_reported = filter_res[filter_res['REDIPortal']]
pos_dict = filter_res_reported['CHROM'].to_dict()
dict_editing_cell_level = {}
for pos in editing_pickle:
    # Only process reported sites
    if pos_dict.get(pos, False):
        for cell in editing_pickle[pos]:
            if cell not in dict_editing_cell_level:
                dict_editing_cell_level[cell] = {
                    'site': 0,
                    'all_ref': 0,
                    'all_edited': 0,
                    'site_ref': 0,
                    'site_edited': 0}
            tp, ref, alt, other = editing_pickle[pos][cell]
            if tp == 3:
                continue
            if tp == 2:
                dict_editing_cell_level[cell]['site'] += 1
                dict_editing_cell_level[cell]['site_ref'] += ref
                dict_editing_cell_level[cell]['site_edited'] += alt
            dict_editing_cell_level[cell]['all_ref'] += ref
            dict_editing_cell_level[cell]['all_edited'] += alt

adata.obs['Edited_Site'] = adata.obs.apply(
    lambda x: dict_editing_cell_level.get(
        x.name)['site'] if x.name in dict_editing_cell_level else 0, axis=1)

adata.obs['Edited_Sites_PK_UMI'] = adata.obs['Edited_Site'] / \
    adata.obs['total_counts'] * 1000

adata.obs['Edited_Read'] = adata.obs.apply(lambda x: dict_editing_cell_level.get(
    x.name)['all_edited'] if x.name in dict_editing_cell_level else 0, axis=1)

adata.obs['Edited_Read_Norm'] = adata.obs['Edited_Read'] / \
    adata.obs['total_counts'] * 1000

adata.obs['Cell_Editing_Indexer'] = adata.obs.apply(lambda x: (dict_editing_cell_level[x.name]['all_edited'] /
                                                               (dict_editing_cell_level[x.name]['all_edited'] +
                                                                dict_editing_cell_level[x.name]['all_ref'])) if x.name in dict_editing_cell_level else 0, axis=1)
adata.obs['CEI'] = np.log(
    adata.obs['Edited_Sites_PK_UMI'] + 1)
for gene in markers:
    adata.obs[f'gRNA_{gene}'] = adata.obs.apply(
        lambda x: x['gRNA'] if gene in x['gRNA'] else 'Control' if 'Control' in x['gRNA'] else np.nan,
        axis=1).astype("category")
    adata.obs[f'gRNA_{gene}'] = adata.obs[f'gRNA_{gene}'].cat.add_categories([
                                                                             'Others'])
    adata.obs[f'gRNA_{gene}'].fillna('Others', inplace=True)
adata_editing = res_to_edited_matrix(editing_pickle)
adata_editing = adata_editing[adata_editing.obs.index.isin(adata.obs.index)]
adata_editing.obs = adata_editing.obs.join(adata.obs)
editing_features = filter_res.sort_values('EDITED', ascending=False).head(2000)
adata_editing = adata_editing[:, editing_features.index]

In [ ]:
from pandarallel import pandarallel
from pysam import VariantFile
filter_res = pd.read_csv(
    editing_tsv,
    sep='\t')
filter_res = filter_res.drop_duplicates()
filter_res = filter_res[filter_res['FILTER_PASS']]
filter_res['index'] = filter_res['CHROM'] + ':' + \
    filter_res['POS'].astype(str) + ':' + '.'
filter_res.set_index('index', inplace=True)

with gzip.open('known_editing_site.pickle.gz', 'rb') as f:
    known_editing_site = pickle.load(f)

In [ ]:
import venn
labels = venn.get_labels(
    [set(list(filter_res['CHROM'] + ':' +
              filter_res['POS'].astype(str))), set(known_editing_site['rediportal'].keys())], fill=['number'])
fig, ax = venn.venn2(
    labels, names=[
        'Editing Sites Found by sCREDIT-Seq', 'Editing Sites in REDIPortal'])
plt.savefig('figures/venn_editing_site.pdf', bbox_inches='tight', dpi=600)
filter_res_reported = filter_res[filter_res['REDIPortal']]
pos_dict = filter_res['CHROM'].to_dict()
plt.figure(figsize=(10, 5))
sns.histplot(filter_res[filter_res.LOC.isin(['ncRNA_exonic', 'intergenic', 'ncRNA_intronic', 'exonic', 'UTR3',
                                             'intronic', 'upstream', 'UTR5', 'downstream',])], x='LOC', palette='tab10')
plt.xticks(rotation=90)
plt.savefig('figures/known_editing_site_location.pdf', bbox_inches='tight')
plt.show()
plt.figure(figsize=(10, 5))
sns.histplot(filter_res[filter_res.REPEAT_TYPE.isin(
    ['SINE', 'LTR', 'Simple_repeat', 'LINE', 'srpRNA', 'DNA'])], x='REPEAT_TYPE', palette='tab10')
plt.xticks(rotation=90)
plt.savefig('figures/known_editing_site_repeat_type.pdf', bbox_inches='tight')
plt.show()

In [ ]:
adata_lda = adata.copy()

adata_lda.obsm["X_pca"] = adata_lda.uns["mixscape_lda"]
sc.pp.neighbors(
    adata_lda,
    n_pcs=6,
    use_rep='X_pca',
    n_neighbors=10)
sc.tl.paga(adata_lda, groups='gene_target')
sc.pl.paga(adata_lda, color='gene_target', plot=False)
sc.tl.umap(adata_lda, init_pos='paga')
sc.tl.tsne(adata_lda, n_pcs=6)
sc.pl.pca(adata_lda, color=["gene_target",
                            'Edited_Sites_PK_UMI_Norm'], cmap='Reds')
sc.pl.tsne(
    adata_lda,
    color=[
        "gene_target",
        'Edited_Sites_PK_UMI_Norm'], cmap='Reds')
sc.pl.umap(adata_lda, color=["gene_target",
                             'Edited_Sites_PK_UMI_Norm'], cmap='Reds')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

dotplot = sc.pl.DotPlot(
    adata,
    groupby="gRNA",
    var_names=['ADAR', 'AIMP1', 'DDX39B', 'DHX9', 'ILF3', 'SLTM', 'TPR'],
    standard_scale="var",  # standard scale: normalize each gRNA to range from 0 to 1
)
colors = ["#854b7a", "#e0d1b9",]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
plt.figure(figsize=(5, 8))
dot_color_df = dotplot.dot_color_df
dot_color_df = dot_color_df.loc[[
    'kd-ADAR-1',
    'kd-ADAR-2', 'kd-ILF3-1',
    'kd-ILF3-2', 'kd-DHX9-1',
    'kd-DHX9-2',
    'kd-AIMP1-1',
    'kd-AIMP1-2',
    'kd-SLTM-1',
    'kd-SLTM-2',
    'kd-TPR-1',
    'kd-TPR-2',
    'kd-DDX39B-1',
    'kd-DDX39B-2',
    'kd-Control-1',
    'kd-Control-2',
    'kd-Control-3',
    'kd-Control-4',
]]
dot_color_df = dot_color_df[['ADAR', 'ILF3',
                             'DHX9', 'AIMP1', 'SLTM', 'TPR', 'DDX39B']]
normalized_df = dot_color_df.apply(lambda x: (x - x.mean()) / x.std())
sns.heatmap(
    normalized_df,
    xticklabels=True,
    yticklabels=True,
    cmap=cmap
)
plt.yticks(fontsize=10)
plt.xlabel('Gene Expression')
plt.ylabel('Knock Down')
plt.savefig('figures/dotplot_heatmap.pdf', bbox_inches='tight')
plt.show()

In [ ]:
sc.pl.violin(
    adata,
    keys='IFIH1',
    groupby='gRNA',
    jitter=0.4,
    multi_panel=True, show=False)
plt.xticks(rotation=90)
plt.savefig('figures/violin_MDA5.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Sample Cell and Test
import scipy
adata.obs['CEI'] = adata.obs['Edited_Sites_PK_UMI']
samp_df = pd.DataFrame()
res_df_p = []
res_df_stat = []
ddx39b_obs = adata.obs[adata.obs.gene_target == 'NT']
control_obs = adata.obs[adata.obs.gene_target == 'NT']
for sample_size in [10, 20, 30, 40, 50, 75, 100, 200, 300]:
    for repeat in range(100):
        tmp_ddx39b_level = ddx39b_obs.sample(sample_size)['CEI']
        tmp_control_level = control_obs.sample(sample_size)['CEI']
        ttest_level = scipy.stats.ttest_ind(
            tmp_ddx39b_level, tmp_control_level)
        res_df_p.append([sample_size, ttest_level.pvalue, 'CEI', repeat])
        res_df_stat.append(
            [sample_size, ttest_level.statistic, 'CEI', repeat])
res_df_p = pd.DataFrame(
    res_df_p,
    columns=[
        'Sample_Size',
        'P_Value',
        'Type',
        'Repeat'])
res_df_p['-log10 P'] = -np.log10(res_df_p['P_Value'])
res_df_stat = pd.DataFrame(
    res_df_stat,
    columns=[
        'Sample_Size',
        'Statistic',
        'Type', 'Repeat'])

plt.figure()
sns.boxplot(
    x="Sample_Size",
    y="-log10 P",
    data=res_df_p,
    flierprops={
        'marker': 'o',
        'markersize': 1,
        'linewidth': 0.1,
        'color': 'black'},
    linewidth=0.5, fill=False)
x_value = -np.log(0.05)
plt.axhline(y=x_value, color='grey', linestyle='--')
plt.ylim(0, 60)
plt.savefig('figures/ttest_p_value_boxplot_nt.pdf', bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
adata.obs['CEI'] = adata.obs['Edited_Sites_PK_UMI']
palette = [
    '#717171',
    '#7f7f7f',
    '#a5a5a5',
    '#cbcbcb',
    '#598351',
    '#72a968',
    '#165581',
    '#1f77b4',
    '#ad3c37',
    '#c45b56',
    '#68929e',
    '#a7dfff',
    '#006374',
    '#00839a',
    '#7e789a',
    '#9d95c0',
    '#a66a95',
    '#f29ad9',
]
plt.figure(figsize=(4, 4))
sns.set_theme(style="ticks")
ax = sns.boxplot(
    adata.obs,
    x="gRNA",
    y="CEI",
    palette=palette, flierprops={'marker': 'o', 'markersize': 1, 'linewidth': 0.1, 'color': 'black', 'alpha': 1}, linewidth=0.5, showfliers=True,)
plt.xticks(rotation=90)
plt.ylabel('Cell Editing Indexer')
plt.ylim(0, 25)
line = adata.obs[adata.obs['gene_target'] == 'NT'].CEI.median()
plt.axhline(line, color='red', linestyle='--', linewidth=0.8)
plt.savefig(
    'boxplot_Cell_Editing_Indexer_CI.pdf',
    bbox_inches='tight')

In [ ]:
import json
from scipy.stats import ttest_ind
with open('boxplot_Cell_Editing_Indexer_A2.txt', 'r') as f:
    A2 = json.load(f)
a2 = list(A2.values())
ci = adata.obs[adata.obs['gene_target'] == 'NT'].CEI.to_list()
t_stat, p_value = ttest_ind( a2,ci)
t_stat, p_value = ttest_ind( adata.obs[adata.obs['gene_target'] == 'NT'].CEI.to_list(
), adata.obs[adata.obs['gene_target'] == 'ADAR'].CEI.to_list())
print(f"T-statistic: {t_stat}, P-value: {p_value}")
t_stat, p_value = ttest_ind(adata.obs[adata.obs['gene_target'] == 'NT'].CEI.to_list(
), adata.obs[adata.obs['gene_target'] == 'ILF3'].CEI.to_list())
print(f"T-statistic: {t_stat}, P-value: {p_value}")

adata.obs[adata.obs['gene_target'] == 'ILF3'].CEI.mean(
) / adata.obs[adata.obs['gene_target'] == 'NT'].CEI.mean()

In [ ]:
from scipy.stats import ttest_ind

ttest_ind(
    adata.obs[adata.obs.gene_target == 'NT']['CEI'], adata.obs[adata.obs.gene_target == 'DDX39B']['CEI'])

In [ ]:
sns.histplot(adata_editing.obs['CEI'])

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
colors = ["#165581",
          "#ffffff", "#f44e28",]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
adata_editing = sc.read('adata_editing_final.h5ad')
adata_editing.obs['CEI'] = adata.obs['Edited_Sites_PK_UMI']
mean = adata_editing.obs['CEI'].mean()
std = adata_editing.obs['CEI'].std()
adata_editing.obs['Relative_Editing_Level'] = (
    adata_editing.obs['CEI'] - mean) / std
sc.pp.log1p(adata_editing)
lda = LinearDiscriminantAnalysis(n_components=6)
adata_editing.obsm['X_lda'] = lda.fit_transform(
    adata_editing.X.toarray(), adata_editing.obs['gene_target'])
adata_editing.obsm['X_pca'] = adata_editing.obsm['X_lda']
sc.pp.neighbors(adata_editing, n_pcs=6, use_rep='X_pca', n_neighbors=15)
sc.pl.paga(adata_editing, color=['gene_target'], plot=False)
sc.tl.umap(adata_editing, init_pos='paga')
sc.pl.umap(
    adata_editing,
    cmap=cmap,
    color=['Relative_Editing_Level', 'gRNA'], show=False, palette=palette, vmin=-2, vmax=2)
plt.savefig('figures/umap_CEI_editing.pdf', bbox_inches='tight', dpi=600)

In [ ]:
adata1 = sc.read_h5ad('adata_final.h5ad')
lda = LinearDiscriminantAnalysis(n_components=6)
adata1.obs['CEI'] = adata1.obs['Edited_Sites_PK_UMI']
mean = adata1.obs['CEI'].mean()
std = adata1.obs['CEI'].std()
adata1.obs['Relative_Editing_Level'] = (
    adata1.obs['CEI'] - mean) / std
adata1 = adata1[:, adata1.var.highly_variable]
adata1.obsm['X_lda'] = lda.fit_transform(
    adata1.X.toarray(), adata1.obs['gene_target'])
adata1.obsm['X_pca'] = adata1.obsm['X_lda']
sc.pp.neighbors(adata1, n_pcs=6, use_rep='X_pca', n_neighbors=15)
sc.tl.paga(adata1, groups='gene_target')
sc.pl.paga(adata1, color=['gene_target'], plot=False)
sc.tl.umap(adata1, init_pos='paga')
sc.pl.umap(
    adata1,
    cmap=cmap,
    color=['Relative_Editing_Level',
           'gRNA'], show=False, palette=palette, vmin=-2, vmax=2)
plt.savefig('figures/umap_CEI_editing_expression.pdf', bbox_inches='tight', dpi=600)

In [ ]:
filter_res_reported = filter_res[filter_res['REDIPortal']]
pos_dict = filter_res_reported['CHROM'].to_dict()
plt.figure(figsize=(10, 5))
sns.histplot(filter_res_reported[filter_res_reported.LOC.isin(['ncRNA_exonic', 'intergenic', 'ncRNA_intronic', 'exonic', 'UTR3',
                                                               'intronic', 'upstream', 'UTR5', 'downstream',])], x='LOC', palette='tab10')
plt.xticks(rotation=90)
plt.savefig('figures/known_editing_site_location.pdf', bbox_inches='tight')
plt.show()
plt.figure(figsize=(10, 5))
sns.histplot(filter_res_reported[filter_res_reported.REPEAT_TYPE.isin(
    ['SINE', 'LTR', 'Simple_repeat', 'LINE', 'srpRNA', 'DNA'])], x='REPEAT_TYPE', palette='tab10')
plt.xticks(rotation=90)
plt.savefig('figures/known_editing_site_repeat_type.pdf', bbox_inches='tight')
plt.show()

In [ ]:
import tqdm
adata_editing = sc.read('adata_editing_all_sites.h5ad')
adata_editing.X = csr_matrix(adata_editing.X)
pos_dict = filter_res_reported['STRAND'].to_dict()
res_df = []
df = pd.DataFrame(
    adata_editing.X.toarray(),
    columns=adata_editing.var_names,
    index=adata_editing.obs_names)
rows, cols = df.shape
pbar = tqdm.tqdm(total=rows)
for index, row in df.iterrows():
    pbar.update(1)
    for col in df.columns:
        if row[col] != 0 and pos_dict.get(col, False):
            res_df.append([index, col, filter_res_reported.loc[col]['LOC'],
                           filter_res_reported.loc[col]['REPEAT_TYPE'], filter_res_reported.loc[col]['GENE']])
res_df = pd.DataFrame(
    res_df, columns=['Cell', 'Site', 'LOC', 'REPEAT_TYPE', 'GENE'])
grna_dict = adata_editing.obs.gRNA.to_dict()
res_df['gRNA'] = res_df['Cell'].map(grna_dict)
res_df.to_pickle('editing_site_info.pkl.gz', compression='gzip')

In [ ]:
res_df = pd.read_pickle('editing_site_info.pkl.gz', compression='gzip')
dict_res = {}
for g, df in res_df.groupby('gRNA'):
    dict_res[g] = {}
    dict_res[g]['Site'] = df.drop_duplicates('Site').shape[0]
    dict_res[g]['Cell'] = df.drop_duplicates('Cell').shape[0]
    dict_res[g]['LOC'] = df.drop_duplicates('Site').LOC.value_counts()
    dict_res[g]['REPEAT_TYPE'] = df.drop_duplicates(
        'Site').REPEAT_TYPE.value_counts()
    dict_res[g]['GENE'] = df.drop_duplicates('Site').GENE.value_counts()
for g in dict_res:
    dict_res[g]['Site_Per_Cell'] = dict_res[g]['Site'] / dict_res[g]['Cell']
for g in dict_res:
    categories_to_merge_loc = list(dict_res[g]['LOC'].index)
    categories_to_merge_loc = [
        i for i in categories_to_merge_loc if i not in [
            'intronic', 'intergenic', 'UTR3']]
    df = dict_res[g]['LOC']
    sum_others = df[df.index.isin(categories_to_merge_loc)].sum()
    df = df[~df.index.isin(categories_to_merge_loc)]
    df['Others'] = sum_others
    dict_res[g]['LOC'] = df
melt_df = []
for g in dict_res:
    df = dict_res[g]['LOC']
    melt_df.append([g, df.intergenic, df.intronic,
                   df.UTR3, df.Others])
melt_df = pd.DataFrame(
    melt_df,
    columns=[
        'gRNA',
        'intergenic',
        'intronic',
        'UTR3',
        'Others'])
melt_df.set_index('gRNA', inplace=True)

In [ ]:
from matplotlib.ticker import PercentFormatter

melt_df = melt_df.loc[grade_order]
melt_df['total'] = melt_df.sum(axis=1)
for column in ['intronic', 'intergenic', 'UTR3', 'Others']:
    melt_df[column] = melt_df[column] / melt_df['total']
melt_df.drop('total', axis=1, inplace=True)
melt_df.plot(
    kind='bar',
    stacked=True,
    color=[
        '#736d8d',
        '#9d95c0',
        '#a67e87',
         '#b0a4af',], width=0.8)
plt.ylim(0, 1)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.xlabel('gRNA')
plt.ylabel('Proportion')
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.savefig(
    'figures/editing_site_location_proportion.pdf',
    bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
repeat_df_plot = filter_res_reported.drop_duplicates().LOC.value_counts()
categories_to_merge_loc = list(repeat_df_plot.index)
categories_to_merge_loc = [
    i for i in categories_to_merge_loc if i not in [
        'intronic', 'intergenic', 'UTR3']]
sum_others = repeat_df_plot[repeat_df_plot.index.isin(
    categories_to_merge_loc)].sum()
repeat_df_plot = repeat_df_plot[~repeat_df_plot.index.isin(
    categories_to_merge_loc)]  # 移除需要合并的行
repeat_df_plot['Others'] = sum_others
repeat_df_plot = repeat_df_plot.sort_values(ascending=False)
labels = repeat_df_plot.index
sizes = repeat_df_plot
colors = ['#c297ca', '#927399', '#a984b1', '#cda0d7',]

plt.pie(
    sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140, textprops={'size': 10})
plt.savefig(
    'figures/editing_site_loc_type_proportion.pdf',
    bbox_inches='tight')

In [ ]:
import matplotlib.pyplot as plt
repeat_df_plot = res_df.drop_duplicates('LOC').REPEAT_TYPE.value_counts()
categories_to_merge_loc = list(repeat_df_plot.index)
categories_to_merge_loc = [
    i for i in categories_to_merge_loc if i not in [
        'SINE']]
sum_others = repeat_df_plot[repeat_df_plot.index.isin(
    categories_to_merge_loc)].sum()
repeat_df_plot = repeat_df_plot[~repeat_df_plot.index.isin(
    categories_to_merge_loc)]  # 移除需要合并的行
repeat_df_plot['Others'] = sum_others
repeat_df_plot = repeat_df_plot.rename({'SINE': 'SINEs/Alu'}, axis=0)

labels = repeat_df_plot.index
sizes = repeat_df_plot
colors = ['#927399', '#b0a4af']


plt.pie(
    sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140, textprops={'size': 12})
plt.savefig(
    'figures/editing_site_repeat_type_proportion.pdf',
    bbox_inches='tight')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
colors = ["#017b7c",
          "#ffffff", "#58784b",]
cmap1 = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
adata1 = sc.read_h5ad('adata_final.h5ad')
adata1.obs['CEI'] = adata1.obs['Edited_Sites_PK_UMI']
mean = adata1.obs['CEI'].mean()
std = adata1.obs['CEI'].std()
adata1.obs['Relative_Editing_Level'] = (
    adata1.obs['CEI'] - mean) / std
adata1.obs = adata1.obs.join(pd.DataFrame(
    adata.X,
    columns=adata.var_names,
    index=adata.obs_names)['DDX39B'], how='left')
mean = adata1.obs['DDX39B'].mean()
std = adata1.obs['DDX39B'].std()
adata1.obs['Relative_DDX39B_Expression_Level'] = (
    adata1.obs['DDX39B'] - mean) / std


adata1 = adata1[adata1.obs.gene_target.isin(['NT', 'DDX39B'])]
adata1.X = adata1.layers['raw'].toarray()
sc.pp.normalize_total(adata1)
sc.pp.log1p(adata1)
sc.pp.regress_out(adata1, ['G2M_score', 'S_score'])
sc.pp.pca(adata1)
sc.pp.neighbors(adata1, n_neighbors=10)
sc.tl.paga(adata1, groups='gene_target')
sc.pl.paga(adata1, color='gene_target', plot=False)
sc.tl.umap(adata1, init_pos='paga')
sc.pl.umap(
    adata1,
    cmap=cmap,
    color=['Relative_Editing_Level',
           'gRNA'], show=False, palette=[
        '#717171',
        '#7f7f7f',
        '#a5a5a5',
        '#cbcbcb',
        '#a66a95',
        '#f29ad9',
    ], vmin=-2, vmax=2)
plt.savefig(
    'figures/umap_DDX39B_editing_expression_1.pdf',
    bbox_inches='tight',
    dpi=600)
sc.pl.umap(
    adata1,
    cmap=cmap1,
    color=['Relative_DDX39B_Expression_Level',
           'gRNA'], show=False, palette=[
        '#717171',
        '#7f7f7f',
        '#a5a5a5',
        '#cbcbcb',
        '#a66a95',
        '#f29ad9',
    ], vmin=-2, vmax=2)
plt.savefig(
    'figures/umap_DDX39B_editing_expression_2.pdf',
    bbox_inches='tight',
    dpi=600)

In [ ]:
adata1 = sc.read_h5ad('adata_final.h5ad')
adata1 = adata1[adata1.obs.gene_target.isin(['NT', 'ADAR'])]
adata1.X = adata1.layers['raw'].toarray()
sc.pp.normalize_total(adata1)
sc.pp.log1p(adata1)
sc.pp.regress_out(adata1, ['G2M_score', 'S_score'])
sc.pp.pca(adata1)
sc.pp.neighbors(adata1, n_neighbors=10)
sc.tl.paga(adata1, groups='gene_target')
sc.pl.paga(adata1, color='gene_target', plot=False)
sc.tl.umap(adata1, init_pos='paga')
adata1.obs['CEI'] = adata1.obs['Edited_Sites_PK_UMI']
mean = adata1.obs['CEI'].mean()
std = adata1.obs['CEI'].std()
adata1.obs['Relative_Editing_Level'] = (
    adata1.obs['CEI'] - mean) / std
sc.pl.umap(
    adata1,
    cmap=sns.diverging_palette(220, 20, as_cmap=True),
    color=['Relative_Editing_Level',
           'gRNA'], show=False, palette=palette)
plt.savefig(
    'figures/umap_ADAR_editing_expression.pdf',
    bbox_inches='tight',
    dpi=600)

In [ ]:
adata = adata_editing
adata.obs['ct'] = adata.obs.apply(
    lambda x: 'kd-Control' if 'Control' in x.gRNA else x.gRNA, axis=1)
sorted(adata.obs.ct.unique())
sc.tl.rank_genes_groups(adata, 'ct', groups=['kd-ADAR-1',
                                             'kd-ADAR-2', 'kd-ILF3-1',
                                             'kd-ILF3-2', 'kd-DHX9-1',
                                             'kd-DHX9-2',
                                             'kd-AIMP1-1',
                                             'kd-AIMP1-2',
                                             'kd-SLTM-1',
                                             'kd-SLTM-2',
                                             'kd-TPR-1',
                                             'kd-TPR-2',
                                             'kd-DDX39B-1',
                                             'kd-DDX39B-2',
                                             ], reference='kd-Control', method='wilcoxon')
# sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)
names = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
scores = pd.DataFrame(adata.uns['rank_genes_groups']['scores'])
pvals = pd.DataFrame(adata.uns['rank_genes_groups']['pvals'])
pvals_adj = pd.DataFrame(adata.uns['rank_genes_groups']['pvals_adj'])
log_fold_changes = pd.DataFrame(
    adata.uns['rank_genes_groups']['logfoldchanges'])
df_dict = {}
for col in names.columns:
    df_dict[col] = pd.concat([names[col], scores[col],
                              pvals[col], pvals_adj[col], log_fold_changes[col]], axis=1)
    df_dict[col].columns = [
        'gene',
        'score',
        'pvals',
        'pvals_adj',
        'log_fold_change']
res_df = []
for k in df_dict.keys():
    df = df_dict[k]
    df = df[(df['pvals_adj'] <= 0.05) & (df['log_fold_change'].abs() >= 0.5)]
    res_df.append([k, df[(df['pvals_adj'] <= 0.05) & (
        df['log_fold_change'].abs() >= 0.5)].shape[0], df[(df['pvals_adj'] <= 0.05) & (
            df['log_fold_change'] >= 0.5)].shape[0], df[(df['pvals_adj'] <= 0.05) & (
                df['log_fold_change'] <= -0.5)].shape[0]])
res_df = pd.DataFrame(res_df, columns=['group', 'n_genes', 'up', 'down'])
# res_df.set_index('group', inplace=True)
# res_df = res_df.loc[
sns.barplot(data=res_df, x='group', y='n_genes', palette=palette[4:])
plt.xticks(rotation=90)
plt.ylabel('Number of DEGs')
plt.xlabel('gRNA')